In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt').read().splitlines()
print(words[:8])
print(len(words))

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']
32033


In [25]:
chars = sorted(set(''.join(words)))
s_to_i = {s:i+1 for i,s in enumerate(chars)}
s_to_i['.'] = 0
i_to_s = {i+1:s for i,s in enumerate(chars)}
i_to_s[0] = '.'

In [ ]:
# i_to_s1 = {i+1:s for i,s in enumerate(chars)}
# i_to_s2 = {i:s for s,i in s_to_i.items()}

# print(i_to_s1)
# print(i_to_s2)
# i_to_s1 == i_to_s2

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


True

Build the dataset

In [ ]:
block_size = 3 #context length: number of chars do we take to predict the next one
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0]*block_size
    for c in w + '.':
        ix = s_to_i[c]
        X.append(context)
        Y.append(ix)
        print(''.join(i_to_s[i] for i in context), '--->', i_to_s[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

print(X,Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .
tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],


In [ ]:
# block_size = 3 #context length: number of chars do we take to predict the next one
# X, Y = [], []
# for w in words[]:
#     context = [0]*block_size
#     for c in w + '.':
#         ix = s_to_i[c]
#         X.append(context)
#         Y.append(ix)
#         context = context[1:] + [ix]

# X = torch.tensor(X)
# Y = torch.tensor(Y)

In [35]:
C = torch.randn(27,2)
C

tensor([[ 0.4604, -0.3841],
        [ 0.1356, -0.6385],
        [-0.2575,  0.1828],
        [-1.1195,  0.0883],
        [ 0.8663, -0.5482],
        [-1.0041,  0.2904],
        [-1.0070, -2.3170],
        [ 0.4037,  0.5119],
        [ 2.4668, -2.0658],
        [ 0.1285, -1.0569],
        [ 0.3842, -1.5807],
        [-1.2396,  0.7150],
        [ 0.5643,  1.1903],
        [-1.3303,  1.4986],
        [ 0.1833, -0.1089],
        [-0.1209,  1.0514],
        [ 0.2814,  0.9989],
        [ 0.4262, -0.1158],
        [ 0.4951, -2.4736],
        [ 0.5132,  1.4697],
        [-1.2437, -0.2826],
        [-0.2650, -0.0033],
        [ 1.0289,  1.8853],
        [ 0.9488, -0.4665],
        [ 0.2649, -0.5232],
        [-0.9316, -1.7019],
        [-0.6356, -0.0966]])

In [ ]:
C[5] #same as F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-1.0041,  0.2904])

In [ ]:
C[[[(0,0,5), (0,5,1), (0,0,0)]]] #embeddings for ..e, .ea, ... resp

tensor([[[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [-1.0041,  0.2904]],

        [[ 0.4604, -0.3841],
         [-1.0041,  0.2904],
         [ 0.1356, -0.6385]],

        [[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [ 0.4604, -0.3841]]])

In [ ]:
C[X] #shape is 32,3,2 (32 data points like ..., ..e, .em, emm, etc.; 3 is the context like ...; 2 is the dimension of each character)
#C[5] would be the embedding for e
#C[[]]

tensor([[[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [ 0.4604, -0.3841]],

        [[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [-1.0041,  0.2904]],

        [[ 0.4604, -0.3841],
         [-1.0041,  0.2904],
         [-1.3303,  1.4986]],

        [[-1.0041,  0.2904],
         [-1.3303,  1.4986],
         [-1.3303,  1.4986]],

        [[-1.3303,  1.4986],
         [-1.3303,  1.4986],
         [ 0.1356, -0.6385]],

        [[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [ 0.4604, -0.3841]],

        [[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [-0.1209,  1.0514]],

        [[ 0.4604, -0.3841],
         [-0.1209,  1.0514],
         [ 0.5643,  1.1903]],

        [[-0.1209,  1.0514],
         [ 0.5643,  1.1903],
         [ 0.1285, -1.0569]],

        [[ 0.5643,  1.1903],
         [ 0.1285, -1.0569],
         [ 1.0289,  1.8853]],

        [[ 0.1285, -1.0569],
         [ 1.0289,  1.8853],
         [ 0.1285, -1.0569]],

        [[ 1.0289,  1

In [71]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [72]:
W1 = torch.rand((6,100)) #6 cuz 3 context * 2 dimensions
b1 = torch.rand(100)

In [92]:
##we need to convert emb into something that we can multiply with W1 because we can't do 32,3,2 * 6,100
##emb[:, 0, :], emb[:, 1, :], emb[:, 2, :] #first char, second char, and third char
##torch.unbind(emb, 1) is the same as emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]
# concat_tensors = torch.unbind(emb, 1) 
# shapes = [concat_tensors[i].shape for i in range(3)]
# print(shapes)
# torch.cat(concat_tensors, 1) #along the first dimension and not the zeroth
#we can do the above or simply just do the following
emb.view(32,6)

tensor([[ 0.4604, -0.3841,  0.4604, -0.3841,  0.4604, -0.3841],
        [ 0.4604, -0.3841,  0.4604, -0.3841, -1.0041,  0.2904],
        [ 0.4604, -0.3841, -1.0041,  0.2904, -1.3303,  1.4986],
        [-1.0041,  0.2904, -1.3303,  1.4986, -1.3303,  1.4986],
        [-1.3303,  1.4986, -1.3303,  1.4986,  0.1356, -0.6385],
        [ 0.4604, -0.3841,  0.4604, -0.3841,  0.4604, -0.3841],
        [ 0.4604, -0.3841,  0.4604, -0.3841, -0.1209,  1.0514],
        [ 0.4604, -0.3841, -0.1209,  1.0514,  0.5643,  1.1903],
        [-0.1209,  1.0514,  0.5643,  1.1903,  0.1285, -1.0569],
        [ 0.5643,  1.1903,  0.1285, -1.0569,  1.0289,  1.8853],
        [ 0.1285, -1.0569,  1.0289,  1.8853,  0.1285, -1.0569],
        [ 1.0289,  1.8853,  0.1285, -1.0569,  0.1356, -0.6385],
        [ 0.4604, -0.3841,  0.4604, -0.3841,  0.4604, -0.3841],
        [ 0.4604, -0.3841,  0.4604, -0.3841,  0.1356, -0.6385],
        [ 0.4604, -0.3841,  0.1356, -0.6385,  1.0289,  1.8853],
        [ 0.1356, -0.6385,  1.0289,  1.8

In [93]:
h = emb.view(32,6) @ W1 + b1